# 🧪 Evaluación de Modelos Sustitutos en Benchmarks Sintéticos

Este notebook demuestra cómo usar el módulo de benchmarks sintéticos para evaluar modelos sustitutos (surrogate models). El objetivo es comparar diferentes modelos (GP, Ridge, PLS, etc.) en funciones de test conocidas para entender su comportamiento antes de aplicarlos a datos reales.

## Contenido
1. **Setup**: Configuración inicial
2. **Benchmarks**: Exploración de funciones de test disponibles
3. **Sampling**: Estrategias de muestreo (Sobol, LHS, Grid)
4. **Noise**: Tipos de ruido para evaluar robustez
5. **Evaluación**: Comparación de modelos sustitutos
6. **Métricas**: RMSE, NLPD, cobertura, calibración
7. **Visualización**: Gráficas de resultados

In [ ]:
# Setup - Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Configuración de plots
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Imports del proyecto
import sys
sys.path.insert(0, '..')

# Benchmarks
from src.benchmarks import (
    get_benchmark, list_benchmarks,
    get_sampler,
    get_noise_injector,
    generate_benchmark_dataset,
    generate_multi_benchmark_suite,
)

# Models
from src.models.gp import GPSurrogateRegressor
from src.models.ridge import RidgeSurrogateRegressor
from src.models.pls import PLSSurrogateRegressor
from src.models.dummy import DummySurrogateRegressor

# Metrics & Evaluation
from src.analysis.surrogate_metrics import compute_surrogate_metrics
from src.analysis.benchmark_runner import (
    evaluate_model_on_dataset,
    evaluate_models_on_suite,
    run_quick_benchmark,
)

print("✅ Setup completo!")

---
## 1. 📊 Funciones Benchmark Disponibles

Tenemos funciones de diferentes dimensionalidades y características:

| Función | Dim | Características | Uso |
|---------|-----|-----------------|-----|
| Forrester | 1D | Una variable, visualizable | Test básico de interpolación |
| Branin | 2D | Multimodal (3 mínimos) | Test de modelado 2D |
| Six-Hump Camel | 2D | Multimodal (6 mínimos locales) | Test de complejidad |
| Goldstein-Price | 2D | Muy no lineal | Test de no-linealidad extrema |
| Hartmann-3 | 3D | Suave, multimodal | Test de dimensión media |
| Ishigami | 3D | Interacciones, sensitivity | Test de interacciones |
| Hartmann-6 | 6D | Estándar en BO | Test de dimensión |
| Borehole | 8D | Modelo físico | Test realista |
| Wing Weight | 10D | Modelo de ingeniería | Test de alta dimensión |

In [ ]:
# Ver todos los benchmarks disponibles
print("Benchmarks disponibles:")
for info in list_benchmarks(include_info=True):
    opt = f"{info['optimal_value']:.4f}" if info['optimal_value'] is not None else "N/A"
    print(f"  {info['name']:20s} | dim={info['dim']:2d} | óptimo={opt}")

In [ ]:
# Visualización de Forrester 1D
bench = get_benchmark("forrester")

# Crear grid denso para visualización
X_viz = np.linspace(0, 1, 200).reshape(-1, 1)
y_viz = bench(X_viz)

# Puntos de muestra (Sobol)
sampler = get_sampler("sobol", seed=42)
X_sample = sampler.sample_bounds(10, bench.bounds)
y_sample = bench(X_sample)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(X_viz, y_viz, 'b-', lw=2, label='Forrester 1D (función real)')
ax.scatter(X_sample, y_sample, c='red', s=100, zorder=5, label=f'Muestras Sobol (n={len(X_sample)})')
ax.axhline(bench.optimal_value, color='green', linestyle='--', alpha=0.5, label=f'Óptimo global: {bench.optimal_value:.3f}')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title(f'{bench.name}: Función de test típica en Bayesian Optimization')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visualización de Branin 2D
bench2d = get_benchmark("branin")

# Grid 2D
n_grid = 100
x1 = np.linspace(bench2d.bounds[0][0], bench2d.bounds[0][1], n_grid)
x2 = np.linspace(bench2d.bounds[1][0], bench2d.bounds[1][1], n_grid)
X1, X2 = np.meshgrid(x1, x2)
X_grid = np.column_stack([X1.ravel(), X2.ravel()])
y_grid = bench2d(X_grid).reshape(n_grid, n_grid)

# Muestras Sobol
X_sample_2d = sampler.sample_bounds(30, bench2d.bounds)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Contour
ax = axes[0]
cs = ax.contourf(X1, X2, y_grid, levels=30, cmap='viridis')
ax.scatter(X_sample_2d[:, 0], X_sample_2d[:, 1], c='red', s=50, edgecolor='white', label='Muestras Sobol')
plt.colorbar(cs, ax=ax)
ax.set_xlabel('x₁')
ax.set_ylabel('x₂')
ax.set_title(f'{bench2d.name}: Mapa de contorno')
ax.legend()

# 3D surface
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot_surface(X1, X2, y_grid, cmap='viridis', alpha=0.8)
ax.set_xlabel('x₁')
ax.set_ylabel('x₂')
ax.set_zlabel('f(x)')
ax.set_title(f'{bench2d.name}: Superficie 3D')

plt.tight_layout()
plt.show()

---
## 2. 🎯 Estrategias de Muestreo (DOE)

El muestreo afecta mucho la calidad del surrogate. Comparamos:
- **Sobol**: Secuencias quasi-aleatorias, excelente cobertura (recomendado)
- **LHS**: Latin Hypercube, clásico en ingeniería
- **Grid**: Regular, solo para visualización en 1D/2D
- **Random**: Baseline, peor cobertura

In [ ]:
# Comparar estrategias de muestreo en 2D
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
n_samples = 50

for ax, name in zip(axes, ["sobol", "lhs", "grid", "random"]):
    sampler = get_sampler(name, seed=42)
    X = sampler.sample(n_samples, dim=2)
    
    ax.scatter(X[:, 0], X[:, 1], alpha=0.7)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.set_title(f'{name.upper()} (n={len(X)})')
    ax.set_xlabel('x₁')
    ax.set_ylabel('x₂')

plt.suptitle('Comparación de Estrategias de Muestreo', fontsize=14)
plt.tight_layout()
plt.show()

---
## 3. 📈 Tipos de Ruido

Evaluamos robustez ante diferentes tipos de ruido:
- **Sin ruido**: Test de interpolación pura
- **Gaussiano**: Ruido homoscedástico (varianza constante)
- **Heteroscedástico**: Varianza depende de x (difícil para GPs estándar)

In [ ]:
# Demostrar tipos de ruido en Forrester
bench = get_benchmark("forrester")
X_test = np.linspace(0, 1, 100).reshape(-1, 1)
y_clean = bench(X_test)

noise_types = [
    ("none", {}, "Sin ruido (interpolación)"),
    ("gaussian", {"sigma": 0.3}, "Gaussiano (σ=0.3)"),
    ("heteroscedastic", {"sigma_base": 0.1, "sigma_scale": 0.5}, "Heteroscedástico"),
]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (noise_name, noise_kwargs, title) in zip(axes, noise_types):
    noise = get_noise_injector(noise_name, seed=42, **noise_kwargs)
    y_noisy = noise.add_noise(y_clean.copy(), X_test)
    
    ax.plot(X_test, y_clean, 'b-', lw=2, label='Función real', alpha=0.5)
    ax.scatter(X_test, y_noisy, c='red', s=10, alpha=0.5, label='Observaciones')
    ax.set_title(title)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.legend()

plt.suptitle('Tipos de Ruido en Datos Sintéticos', fontsize=14)
plt.tight_layout()
plt.show()

---
## 4. 🔬 Generación de Dataset Completo

La función `generate_benchmark_dataset` crea un dataset listo para entrenar y evaluar:

In [ ]:
# Generar dataset para Forrester con ruido gaussiano
dataset = generate_benchmark_dataset(
    benchmark="forrester",
    n_train=30,
    n_test=100,
    sampler="sobol",
    noise="gaussian",
    noise_kwargs={"sigma": 0.1},
    seed=42,
)

print(f"Dataset generado: {dataset}")
print(f"\nX_train shape: {dataset.X_train.shape}")
print(f"y_train range: [{dataset.y_train.min():.3f}, {dataset.y_train.max():.3f}]")
print(f"X_test shape: {dataset.X_test.shape}")

---
## 5. 🤖 Entrenamiento y Evaluación de un Modelo

Ejemplo completo con GP en Forrester 1D:

In [ ]:
# Entrenar GP
gp = GPSurrogateRegressor(n_restarts_optimizer=5)
gp.fit(dataset.X_train, dataset.y_train)

# Predicción con incertidumbre
mean_pred, std_pred = gp.predict_dist(dataset.X_test)

# Calcular métricas extendidas
metrics = gp.compute_extended_metrics(dataset.y_test_clean, mean_pred, std_pred)

print("=" * 50)
print("MÉTRICAS DEL MODELO GP")
print("=" * 50)
print(f"\n📊 Precisión:")
print(f"   MAE  = {metrics.mae:.4f}")
print(f"   RMSE = {metrics.rmse:.4f}")
print(f"   R²   = {metrics.r2:.4f}")

print(f"\n🎯 Incertidumbre:")
print(f"   NLPD (menor mejor)    = {metrics.nlpd:.4f}")
print(f"   Coverage 95%          = {metrics.coverage_95:.2%} (ideal: 95%)")
print(f"   Error de calibración  = {metrics.calibration_error_95:.4f}")
print(f"   Ancho medio IC 95%    = {metrics.mean_interval_width_95:.4f}")
print(f"   Sharpness (mean σ)    = {metrics.sharpness:.4f}")

In [ ]:
# Visualización del ajuste GP en 1D
fig, ax = plt.subplots(figsize=(12, 6))

# Ordenar para plotting
idx = np.argsort(dataset.X_test.ravel())
X_sorted = dataset.X_test[idx]
mean_sorted = mean_pred[idx]
std_sorted = std_pred[idx]
y_true_sorted = dataset.y_test_clean[idx]

# Función real
ax.plot(X_sorted, y_true_sorted, 'k-', lw=2, label='Función real', zorder=3)

# Predicción GP
ax.plot(X_sorted, mean_sorted, 'b-', lw=2, label='Predicción GP', zorder=4)

# Bandas de confianza
ax.fill_between(X_sorted.ravel(), 
                mean_sorted - 1.96*std_sorted, 
                mean_sorted + 1.96*std_sorted,
                alpha=0.2, color='blue', label='95% CI')
ax.fill_between(X_sorted.ravel(), 
                mean_sorted - std_sorted, 
                mean_sorted + std_sorted,
                alpha=0.3, color='blue', label='68% CI')

# Puntos de entrenamiento
ax.scatter(dataset.X_train, dataset.y_train, c='red', s=80, 
           edgecolor='white', zorder=5, label=f'Train (n={dataset.n_train})')

ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title(f'GP en {dataset.benchmark_name} | RMSE={metrics.rmse:.4f}, R²={metrics.r2:.3f}, Coverage={metrics.coverage_95:.1%}')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

---
## 6. ⚔️ Comparación de Múltiples Modelos

Comparamos GP, Ridge, PLS y Dummy en el mismo dataset:

In [ ]:
# Definir modelos a comparar
models = {
    "Dummy": DummySurrogateRegressor(),
    "Ridge": RidgeSurrogateRegressor(),
    "PLS (2 comp)": PLSSurrogateRegressor(n_components=1),  # 1 para 1D
    "GP (Matérn 3/2)": GPSurrogateRegressor(n_restarts_optimizer=3),
}

# Evaluar cada modelo
results = {}
for name, model in models.items():
    model.fit(dataset.X_train, dataset.y_train)
    mean, std = model.predict_dist(dataset.X_test)
    metrics = model.compute_extended_metrics(dataset.y_test_clean, mean, std)
    results[name] = {
        "mean": mean,
        "std": std,
        "metrics": metrics
    }

# Tabla comparativa
comparison_data = []
for name, r in results.items():
    m = r["metrics"]
    comparison_data.append({
        "Model": name,
        "RMSE": m.rmse,
        "MAE": m.mae,
        "R²": m.r2,
        "NLPD": m.nlpd if m.nlpd else "N/A",
        "Coverage 95%": f"{m.coverage_95:.1%}" if m.coverage_95 else "N/A",
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n📊 COMPARACIÓN DE MODELOS")
print("="*70)
print(comparison_df.to_string(index=False))

In [ ]:
# Visualización comparativa
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

colors = ['red', 'green', 'orange', 'blue']

for ax, (name, r), color in zip(axes, results.items(), colors):
    idx = np.argsort(dataset.X_test.ravel())
    X_sorted = dataset.X_test[idx]
    mean_sorted = r["mean"][idx]
    y_true_sorted = dataset.y_test_clean[idx]
    
    # Función real
    ax.plot(X_sorted, y_true_sorted, 'k-', lw=2, alpha=0.5, label='Función real')
    
    # Predicción
    ax.plot(X_sorted, mean_sorted, color=color, lw=2, label='Predicción')
    
    # Bandas si hay std
    if r["std"] is not None:
        std_sorted = r["std"][idx]
        ax.fill_between(X_sorted.ravel(), 
                        mean_sorted - 1.96*std_sorted, 
                        mean_sorted + 1.96*std_sorted,
                        alpha=0.2, color=color)
    
    # Training points
    ax.scatter(dataset.X_train, dataset.y_train, c='gray', s=40, alpha=0.5)
    
    m = r["metrics"]
    ax.set_title(f'{name}\nRMSE={m.rmse:.4f}, R²={m.r2:.3f}')
    ax.legend(loc='upper right', fontsize=9)

plt.suptitle(f'Comparación de Modelos en {dataset.benchmark_name}', fontsize=14)
plt.tight_layout()
plt.show()

---
## 7. 🚀 Evaluación en Múltiples Benchmarks

Usando `run_quick_benchmark` para una evaluación rápida:

In [ ]:
# Evaluación rápida en varios benchmarks
models_to_test = {
    "Dummy": DummySurrogateRegressor(),
    "Ridge": RidgeSurrogateRegressor(),
    "PLS": PLSSurrogateRegressor(n_components=2),
    "GP": GPSurrogateRegressor(n_restarts_optimizer=3),
}

suite_results = run_quick_benchmark(
    models=models_to_test,
    benchmarks=["forrester", "branin", "hartmann3"],
    n_train=50,
    n_test=200,
    noise_sigma=0.1,
    seed=42,
    verbose=True
)

In [ ]:
# Ver tabla de resultados
summary_df = suite_results.get_summary_df()
print("\n📊 RESUMEN DE RESULTADOS")
print(summary_df.to_string(index=False))

In [ ]:
# Gráfico de barras por modelo
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE por benchmark
ax = axes[0]
pivot_rmse = summary_df.pivot(index='benchmark', columns='model', values='rmse')
pivot_rmse.plot(kind='bar', ax=ax, width=0.8)
ax.set_title('RMSE por Benchmark (menor es mejor)')
ax.set_ylabel('RMSE')
ax.legend(title='Modelo')
ax.tick_params(axis='x', rotation=45)

# R² por benchmark
ax = axes[1]
pivot_r2 = summary_df.pivot(index='benchmark', columns='model', values='r2')
pivot_r2.plot(kind='bar', ax=ax, width=0.8)
ax.set_title('R² por Benchmark (mayor es mejor)')
ax.set_ylabel('R²')
ax.legend(title='Modelo')
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Ranking final de modelos
print("\n🏆 RANKING FINAL (por RMSE promedio)")
print("="*50)
print(suite_results.get_model_ranking("rmse"))

---
## 8. 📝 Resumen y Conclusiones

### Uso típico:

```python
# 1. Generar dataset
dataset = generate_benchmark_dataset(
    benchmark="branin",  # Elegir función de test
    n_train=50,          # Muestras de entrenamiento
    n_test=200,          # Muestras de test
    sampler="sobol",     # Estrategia de muestreo
    noise="gaussian",    # Tipo de ruido
    noise_kwargs={"sigma": 0.1},
    seed=42,
)

# 2. Entrenar modelo
model = GPSurrogateRegressor()
model.fit(dataset.X_train, dataset.y_train)

# 3. Predecir con incertidumbre
mean, std = model.predict_dist(dataset.X_test)

# 4. Evaluar métricas
metrics = model.compute_extended_metrics(dataset.y_test_clean, mean, std)
print(f"RMSE: {metrics.rmse:.4f}")
print(f"NLPD: {metrics.nlpd:.4f}")
print(f"Coverage 95%: {metrics.coverage_95:.2%}")
```

### Para comparar modelos masivamente:

```python
from src.analysis.benchmark_runner import run_quick_benchmark

results = run_quick_benchmark(
    models={"GP": GPSurrogateRegressor(), "Ridge": RidgeSurrogateRegressor()},
    benchmarks=["forrester", "branin", "hartmann3", "hartmann6"],
)
print(results.get_model_ranking("rmse"))
```

### Métricas importantes:
- **RMSE/MAE**: Precisión de predicción
- **R²**: Varianza explicada
- **NLPD**: Calidad de incertidumbre (penaliza sobre/sub-confianza)
- **Coverage 95%**: ¿El 95% de puntos cae en el IC 95%?
- **Calibration Error**: |Coverage - 0.95|